In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchbraid
import torchbraid.utils
from torch.utils.data import Dataset, DataLoader
import sys

from network_architecture import parse_args, ParallelNet
from mpi4py import MPI

import data


In [3]:
device = 'cuda'

In [74]:
model = ParallelNet(256, 4, 10000, 128,
              local_steps=4,
              ).to(device)


In [75]:
model_posttrain = torch.load('serialnet_trained')

In [70]:
dir(model.parallel_nn.steps)

AttributeError: 'LayerParallel' object has no attribute 'steps'

In [77]:
# Loop through the parallel net
num_layers = 4

# Loop over layers, besides last one
reg_value = 0
for i in range(num_layers - 1):
    # Loop over ** all ** weights
    for param_name, weights_n in model.parallel_nn.local_layers[i].named_parameters():
        # Grab weight from next layer 
        weights_np1 = model.parallel_nn.local_layers[i + 1].state_dict()[param_name]
        print(weights_np1)
        # Calculate norm
        reg_value += torch.norm(weights_n - weights_np1, p='fro')

print(reg_value)
    

tensor([[-0.0005,  0.0335, -0.0514,  ..., -0.0183, -0.0371, -0.0231],
        [-0.0619,  0.0282, -0.0300,  ...,  0.0209, -0.0180,  0.0386],
        [-0.0173, -0.0233,  0.0157,  ..., -0.0438, -0.0135,  0.0542],
        ...,
        [-0.0420,  0.0451,  0.0296,  ..., -0.0102, -0.0336,  0.0337],
        [-0.0194,  0.0076,  0.0554,  ..., -0.0463, -0.0046, -0.0382],
        [-0.0270, -0.0300,  0.0121,  ..., -0.0581,  0.0516,  0.0445]],
       device='cuda:0')
tensor([[-0.0223,  0.0254,  0.0197,  ..., -0.0082, -0.0308,  0.0566],
        [ 0.0606,  0.0331, -0.0030,  ..., -0.0343, -0.0257, -0.0196],
        [-0.0282, -0.0493, -0.0514,  ...,  0.0033, -0.0245, -0.0457],
        ...,
        [-0.0303,  0.0623, -0.0539,  ...,  0.0622, -0.0578, -0.0501],
        [ 0.0031, -0.0105, -0.0459,  ...,  0.0182, -0.0264, -0.0528],
        [-0.0317,  0.0481, -0.0366,  ...,  0.0219, -0.0303, -0.0316]],
       device='cuda:0')
tensor([[ 0.0209, -0.0325,  0.0452,  ...,  0.0197, -0.0466,  0.0448],
        [-0.01

In [73]:
len(model.parallel_nn.local_layers)

4

In [56]:
for x in model.parallel_nn.local_layers[0].named_parameters(): 
    # print(x)
    # print(type(x))
    print(x[0])
    # print(type(x[1]))
    # break

layer.F.sa.heads.0.key.weight
layer.F.sa.heads.0.query.weight
layer.F.sa.heads.0.value.weight
layer.F.sa.heads.1.key.weight
layer.F.sa.heads.1.query.weight
layer.F.sa.heads.1.value.weight
layer.F.sa.heads.2.key.weight
layer.F.sa.heads.2.query.weight
layer.F.sa.heads.2.value.weight
layer.F.sa.heads.3.key.weight
layer.F.sa.heads.3.query.weight
layer.F.sa.heads.3.value.weight
layer.F.sa.proj.weight
layer.F.sa.proj.bias
layer.F.ffwd.net.0.weight
layer.F.ffwd.net.0.bias
layer.F.ffwd.net.2.weight
layer.F.ffwd.net.2.bias
layer.F.ln1.weight
layer.F.ln1.bias
layer.F.ln2.weight
layer.F.ln2.bias


In [40]:
model.parallel_nn.state_dict()[x[0]]

tensor([[-0.0005,  0.0335, -0.0514,  ..., -0.0183, -0.0371, -0.0231],
        [-0.0619,  0.0282, -0.0300,  ...,  0.0209, -0.0180,  0.0386],
        [-0.0173, -0.0233,  0.0157,  ..., -0.0438, -0.0135,  0.0542],
        ...,
        [-0.0420,  0.0451,  0.0296,  ..., -0.0102, -0.0336,  0.0337],
        [-0.0194,  0.0076,  0.0554,  ..., -0.0463, -0.0046, -0.0382],
        [-0.0270, -0.0300,  0.0121,  ..., -0.0581,  0.0516,  0.0445]],
       device='cuda:0')

In [48]:
y = x[0].split('.')
y[1] = str(int(y[1]) + 1)
y = '.'.join(y)
print(y)
print(x[0])

local_layers.1.layer.F.sa.heads.0.key.weight
local_layers.0.layer.F.sa.heads.0.key.weight


In [49]:
model.parallel_nn.state_dict()[x[0]] - model.parallel_nn.state_dict()[y]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')